# `views.py` 복습

## 함수 본문
```python
from django.shortcuts import render, redirect
from .forms import ArticleForm, CommentForm
from django.contrib.auth.decorators import login_required

@login_required
def create(request):
    if request.method == 'POST':
        form = ArticleForm(request.POST)
        if form.is_valid():
            article = form.save(commit=False)
            article.user = request.user
            article.save()
            return redirect('articles:index')

    else:
        form = ArticleForm()
    context = {
        'form': form,
    }
    return render(request, 'create.html', context)
```

### `@login_required`
Django에서 사용되는 뷰 데코레이터로, 해당 뷰에 접근하기 전에 로그인이 되어 있는지 확인하는 기능을 함

- redirect_field_name: 로그인하지 않은 경우 리다이렉트 시 전달할 쿼리 문자열의 파라미터명 (기본값: 'next')

#### 쿼리 문자열과 파라미터란?

- 쿼리 문자열은 URL에서 물음표 `?` 뒤에 위치하며, 여러 개의 `키=값` 형태로 구성
- 예시 : `/accounts/login/?next=/orders/`

#### redirect_field_name의 역할
1. 비로그인 사용자가 접근 시 :
    로그인 페이지로 리다이렉트하면서 원래 이동하려던 url을 next에 저장
2. 로그인 성공 시 :
    next에 저장된 url로 리다이렉트(원래 가고자했던 url로 이동)

#### 본문기준 url 동작 흐름

1. 비로그인 사용자가 /create/에 접근
    > /accounts/login/?next=/create/
    - next=/create/는 로그인 후 복귀할 경로
2. 로그인 성공 후
    > /create/ (원래 접근하려던 경로로 복귀)
3. 성공적으로 글 작성 시
    > 리다이렉트 대상인 articles:index 경로

### `else: form = ArticleForm()`
먼저 빈 폼을 사용자에게 제공하는 else 구문이 먼저 실행

### form에 글을 쓴 뒤
submit을 누르면 POST 요청으로 인해 if문 True가 실행
```python
if request.method == 'POST':
        form = ArticleForm(request.POST)
        if form.is_valid(): # 유효성 검사
            article = form.save(commit=False) # 현재 제출된 form에는 article을 작성한 user_id가 없으므로 대기상태 만듬
            article.user = request.user # article의 user_id 컬럼에 request 들어온 user_id 값 할당
            article.save() # title, content, user_id 모두 저장
            return redirect('articles:index')
```

##### form.is_valid() 통과 후 주의점
`ArticleForm`에서 `user` 필드를 제외하고 `form.is_valid()`를 통과한 후 바로 `article.save()`를 호출하면 DB 저장 단계에서 저장 실패가 발생
- Django의 ForeignKey 기본 설정
    - ForeignKey 필드는 `user = models.ForeignKey(User, on_delete=models.CASCADE)` 라고만 작성해도 `null=False` 조건 적용